In [1]:
import numpy as np
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = ['A', 'B', 'C', 'Five', 'Point', 'V']
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

def cat_to_1hot(cat):
    cat = np.array(cat)
    intvals = label_encoder.transform(cat.reshape(-1,1))
    vals = onehot_encoder.transform(intvals.reshape(-1,1))
    return vals[0]

def onehot_to_cat(y):
    inverted = label_encoder.inverse_transform([argmax(y)])
    return inverted

# test it
tests = ['A', 'A', 'C', 'V']
enc = cat_to_1hot(tests)
inv = onehot_to_cat(enc[0])
print('tests', tests, 'inv', inv, 'enc', enc)

tests ['A', 'A', 'C', 'V'] inv ['A'] enc [1. 0. 0. 0. 0. 0.]


c:\users\x\source\repos\gesture\env64\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\x\source\repos\gesture\env64\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import os

#Original dataset path
ORIGINAL = 'Marcel_Train'
TEST = 'Marcel_Test'
IMG_SIZE = 120
size_images = dict()

def label_img(img):
    word_label = img.split('-')[0]
    arr_label = cat_to_1hot(word_label)
    return arr_label

#create training dataset which includes images and labels

def dir_to_array(dir):
    x_item = []
    y_item = []
    items = []
    
    for img in os.listdir(dir):
        label = label_img(img)
        path = os.path.join(dir, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if(img is not None):
            heigth, width = img.shape[:2]
            size_images[path] = {width, heigth}
            #reshape the images to the same size
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            x_item.append(img)
            y_item.append(label)
            items.append([img, label])
    return items

train_arr = dir_to_array(ORIGINAL)

col_names = ["x", 'label']
df = pd.DataFrame(train_arr, columns=col_names)
train_set = df.values
train_data = train_set[:,0]
train_labels = train_set[:, 1]

eval_arr = dir_to_array(TEST)
df = pd.DataFrame(eval_arr)
eval_set = df.values
eval_data = eval_set[:,0]
eval_labels = eval_set[:,1]
eval_labels = np.array(list(eval_labels), dtype=np.uint8)


print(train_data[0].shape, train_data[0].dtype)
print(train_labels[0:1])
print('eval data', eval_data[0:1])
print('eval labels', eval_labels)

c:\users\x\source\repos\gesture\env64\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(120, 120) uint8
[array([1., 0., 0., 0., 0., 0.])]
eval data [array([[115, 115, 115, ..., 145, 146, 146],
       [115, 115, 115, ..., 145, 146, 146],
       [115, 115, 115, ..., 145, 146, 146],
       ...,
       [106, 106, 106, ..., 156, 157, 157],
       [107, 107, 106, ..., 156, 157, 157],
       [107, 107, 106, ..., 156, 157, 157]], dtype=uint8)]
eval labels [[1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 ...
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]]


In [7]:
import tflearn
import tensorflow as tf
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features, [-1, IMG_SIZE, IMG_SIZE, 1])
    conv1 = tf.layers.conv2d(
        inputs=input_layer, 
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(input=conv1, pool_size=[2, 2], stride=2)
    pool1_flat = tf.reshape(pool1, [-1, 14, 14, 32])
    dense = tf.layers.dense(input=pool1_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.droput(input=dense, rate=0.4, training=(mode == tf.estimator.ModeKeys.TRAIN))
    logits = tf.layers.dense(input=dropout, units=6)
    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')        
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global__step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metrics_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [8]:
def main(arg):
    my_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir='temp/')
    tensors_to_log = {'probabilities': 'softmax_tensor'}
    logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=train_data,
        y=train_labels,
        batch_size=14400,
        num_epochs=None,
        shuffle=True)
    my_classifier.train(input_fn=train_input_fn, steps=len(train_labels), hooks=[logging_hook])
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=eval_data,
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = my_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    

In [5]:
print(train_data[0].shape)


(120, 120)


In [9]:
if __name__ == "__main__":
    tf.app.run()


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'temp/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A919CC6EF0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


ValueError: An initializer for variable conv2d/kernel of <dtype: 'string'> is required